In [76]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series 

In [78]:
data_train=pd.read_csv('review_rating_train.csv')
data_train.head()

,review_text,review_rating
0,"penasaran ama iklannya &amp; review org"" yg ka...",5
1,"kandungan olivenya bagus banget di kulit, paka...",5
2,love this! beneran bantu nipisin spot kaya mis...,5
3,"udah jatuh cinta bgt sama sebamed yg ini, sebe...",5
4,salah satu line dari paulas choice kesukaan sa...,5


In [79]:
data_test=pd.read_csv('review_rating.csv')
data_test.head()

,review_text,review_rating
0,ole flamingo shade kesukaanku! pink fuchsia de...,5
1,"body lotion ber-spf pertamaku dulu, hehe. pake...",5
2,gw sk-2 addict dan gw dikenalkan produk kiehl'...,5
3,pertamanya sih aneh pas diolesin di bibir. kay...,5
4,salah satu facial wash favorit aku. aku pakai ...,5


In [80]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_text    10500 non-null  object
 1   review_rating  10500 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 164.2+ KB


In [81]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_text    4500 non-null   object
 1   review_rating  4500 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.4+ KB


In [82]:
data_train.review_rating.value_counts()

4    3780
5    3570
3    1890
2     945
1     315
Name: review_rating, dtype: int64

In [83]:
data_test.review_rating.value_counts()

4    1620
5    1530
3     810
2     405
1     135
Name: review_rating, dtype: int64

### Proses Klasifikasi tanpa Normalisasi Teks

In [84]:
x_train,y_train=data_train['review_text'],data_train['review_rating']
x_test,y_test=data_test['review_text'],data_test['review_rating']

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
count=cv.fit_transform(x_train)
#count1=pd.DataFrame(count.toarray(),columns=cv.get_feature_names())
#count1.T.sort_values(by=0, ascending=False).head(10)

In [86]:
count_test = cv.transform(x_test)

In [87]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

mnb=MultinomialNB().fit(count,y_train)
datatest=mnb.predict(count_test)
print("Akurasi : ",metrics.accuracy_score(y_test,datatest)*100)

Akurasi :  47.68888888888888


### Proses Klasifikasi dengan Normalisasi Teks

In [88]:
!pip install Sastrawi

In [89]:
import string,csv,re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize

#data train
data_train.head()
data_train['review_text']=data_train['review_text'].str.lower()
data_train['review_text']=data_train['review_text'].str.replace('(https?://[\w\.\/]*)', '')
data_train['review_text']=data_train['review_text'].str.replace('(?:&(?:lt|nbsp|amp|gt);)', '')
data_train['review_text']=data_train['review_text'].str.replace('[^A-Za-z0-9\s\-\/]', '')
data_train['review_text']=data_train['review_text'].str.replace('[0-9]', '')
data_train['review_text']=data_train['review_text'].str.replace('\n', ' ')
data_train['review_text']=data_train['review_text'].str.replace('\s{2,}', ' ')
data_train.dropna(subset=['review_text'], inplace=True)
data_train=data_train.drop_duplicates(['review_text'])

#data test
data_test.head()
data_test['review_text']=data_test['review_text'].str.lower()
data_test['review_text']=data_test['review_text'].str.replace('(https?://[\w\.\/]*)', '')
data_test['review_text']=data_test['review_text'].str.replace('(?:&(?:lt|nbsp|amp|gt);)', '')
data_test['review_text']=data_test['review_text'].str.replace('[^A-Za-z0-9\s\-\/]', '')
data_test['review_text']=data_test['review_text'].str.replace('[0-9]', '')
data_test['review_text']=data_test['review_text'].str.replace('\n', ' ')
data_test['review_text']=data_test['review_text'].str.replace('\s{2,}', ' ')
data_test.dropna(subset=['review_text'], inplace=True)
data_test=data_test.drop_duplicates(['review_text'])

factory = StopWordRemoverFactory()
stopwords_id = factory.get_stop_words()

def delete_stopwords(document):
    stopwords_remover = [text for text in document.split() if text not in stopwords_id]
    stopwords_remover_join = ' '.join(stopwords_remover)

    return stopwords_remover_join

data_train['review_text']=data_train['review_text'].apply(delete_stopwords)
data_test['review_text']=data_test['review_text'].apply(delete_stopwords)

def translate(user_string):
    user_string=user_string.split(" ")
    j = 0
    for _str in user_string:
        filename = "Slang1.csv"
        #print('_str = ',_str)
        with open(filename, "r") as myCSVfile:
            dataFromFile = csv.reader(myCSVfile, delimiter=",")
            for row in dataFromFile:
                #print('row 0 =',row[0])
                if _str.lower() == row[0].lower():
                    #print('ada kata yang perlu dinormalisasi:',_str.lower(),'diubah menjadi:',row[1])
                    user_string[j] = row[1]
                    #print('hasil perubahan =',user_string[j])
                #else:
                    #print('tidak sama, _str ='+_str+', row [0]=',row[0])
            myCSVfile.close()
        j = j + 1
    return ' '.join(user_string)
#print('review text sebelum normalisasi:')
#print(data['review_text'])
data_train['review_text'] = data_train['review_text'].apply(lambda x: translate(x)  )
data_test['review_text'] = data_test['review_text'].apply(lambda x: translate(x)  )
#data['review_text'].to_csv('test.csv')
#print('review text sesudah normalisasi:')
#print(data['review_text'])

token_train=data_train['review_text'].apply(word_tokenize)
token_test=data_test['review_text'].apply(word_tokenize)

In [90]:
x_train,y_train=data_train['review_text'],data_train['review_rating']
x_test,y_test=data_test['review_text'],data_test['review_rating']

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
count=cv.fit_transform(x_train)

In [92]:
count_test = cv.transform(x_test)

In [93]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

mnb=MultinomialNB().fit(count,y_train)
datatest=mnb.predict(count_test)
print("Akurasi : ",metrics.accuracy_score(y_test,datatest)*100)

Akurasi :  47.03522068658047


In [94]:
dataset_baru = pd.DataFrame({'review_text': x_test,
                             'review_rating': datatest})

In [95]:
# Memisahkan dataset berdasarkan ratingnya

data1 = dataset_baru[dataset_baru['review_rating'] == 1]
data2 = dataset_baru[dataset_baru['review_rating'] == 2]
data3 = dataset_baru[dataset_baru['review_rating'] == 3]
data4 = dataset_baru[dataset_baru['review_rating'] == 4]
data5 = dataset_baru[dataset_baru['review_rating'] == 5]

In [96]:
# Mengubah dataframe ke file csv

Combined_Test = [data5,data4,data3,data2,data1]
Together_Test = pd.concat(Combined_Test)

Together_Test.to_csv('review_rating_norm.csv')

In [97]:
dataset_baru1 = pd.DataFrame({'review_text': x_train,
                             'review_rating': y_train})

In [98]:
# Memisahkan dataset berdasarkan ratingnya

data1_1 = dataset_baru1[dataset_baru1['review_rating'] == 1]
data1_2 = dataset_baru1[dataset_baru1['review_rating'] == 2]
data1_3 = dataset_baru1[dataset_baru1['review_rating'] == 3]
data1_4 = dataset_baru1[dataset_baru1['review_rating'] == 4]
data1_5 = dataset_baru1[dataset_baru1['review_rating'] == 5]

In [99]:
# Mengubah dataframe ke file csv

Combined_Train=[data1_5,data1_4,data1_3,data1_2,data1_1]
Together_Train=pd.concat(Combined_Train)

Together_Train.to_csv('review_rating_train_norm.csv')

In [100]:
#Combined=[data1_5,data5,data1_4,data4,data1_3,data3,data1_2,data2,data1_1,data1]
#Together=pd.concat(Combined)
#Together.to_csv('review_rating_all_norm.csv',index=False)